In [2]:
import numpy as np
import tensorflow as tf

In [3]:
# https://github.com/guillaume-chevalier/seq2seq-signal-prediction

In [4]:
voc = 10
n_steps = 14
n_input = 2
n_output = 1

In [61]:
def get_batch(batch_size):
    x = np.random.randint(voc, size=[batch_size,n_steps, n_input])
    y = np.flip(x, axis=1)
    y = np.sum(y, axis=2)
    y = y.reshape((batch_size, n_steps, 1))
    
    seq = np.empty((batch_size), dtype=np.int)
    seq.fill(n_steps)
    return x, y, seq

x, y, seq = get_batch(100)

#print(x[0])
#print(y[0])

print(seq)
'''
print(x.shape)
print(y.shape)
print(x[:, 0, :])
print(y[:, 0])
#print(y[0])
'''

[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14]


'\nprint(x.shape)\nprint(y.shape)\nprint(x[:, 0, :])\nprint(y[:, 0])\n#print(y[0])\n'

In [62]:
n_hidden = 20
layers_stacked_count = 2  # Number of stacked recurrent cells, on the neural depth axis. 

tf.reset_default_graph()

sess = tf.InteractiveSession()

enc_inp = tf.placeholder(tf.float32, [None, n_steps, n_input])
expect = tf.placeholder(tf.float32, [None, n_steps, 1])
expect_length = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32, [])

cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(n_hidden), output_keep_prob=keep_prob) for i in range(layers_stacked_count)]
cell = tf.contrib.rnn.MultiRNNCell(cells)

encoded_outputs, encoded_states = tf.nn.dynamic_rnn(cell, enc_inp, dtype=tf.float32)

helper = tf.contrib.seq2seq.TrainingHelper(expect, expect_length)

de_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(n_hidden), output_keep_prob=keep_prob) for i in range(layers_stacked_count)]
de_cell = tf.contrib.rnn.MultiRNNCell(de_cells)

decoder = tf.contrib.seq2seq.BasicDecoder(cell=de_cell, helper=helper, initial_state=encoded_states)

decoder_outputs, final_decoder_state = tf.contrib.seq2seq.dynamic_decode(decoder)

decoder_logits = decoder_outputs.rnn_output

h = tf.contrib.layers.fully_connected(decoder_logits, n_output)

diff = tf.squared_difference(h, expect)
batch_loss = tf.reduce_sum(diff, axis=1)
loss = tf.reduce_mean(batch_loss)

optimiser = tf.train.AdamOptimizer(1e-3)
training_op = optimiser.minimize(loss)

init_op = tf.global_variables_initializer()
    

In [66]:
sess = tf.InteractiveSession()

init_op.run()

for e in range(10):
    for i in range(100):
        batch_x, batch_y ,seq = get_batch(50)
        training_op.run(feed_dict={enc_inp:batch_x, expect: batch_y, expect_length:seq, keep_prob:0.5})
        
    print(loss.eval(feed_dict={enc_inp:batch_x, expect: batch_y, expect_length:seq, keep_prob:1}))

280.096
219.746
221.117
115.689
53.676
37.8953
24.4736
19.1247
19.1786
14.2717


In [67]:
test_x, test_y, seq = get_batch(2)
print(test_y[0])
print('---------')
feed_dict = {enc_inp:test_x, expect:test_y, expect_length:seq, keep_prob:1}
pred = h.eval(feed_dict=feed_dict)

print(pred[0])



[[15]
 [10]
 [ 5]
 [17]
 [15]
 [ 9]
 [ 8]
 [16]
 [ 7]
 [ 4]
 [ 8]
 [ 7]
 [ 6]
 [ 8]]
---------
[[ 13.11734104]
 [  9.93261909]
 [  4.59088039]
 [ 13.38347816]
 [ 13.1373806 ]
 [  8.84623623]
 [  7.51879263]
 [ 13.3497324 ]
 [  6.21972895]
 [  3.92029834]
 [  7.83621931]
 [  6.41021967]
 [  5.42207432]
 [  7.76618719]]


In [64]:
for e in range(20):
    for i in range(100):
        batch_x, batch_y = get_batch(50)
        feed_dict = {enc_inp[t]:batch_x[t] for t in range(len(enc_inp))}
        feed_dict.update({expect[t]: batch_y[t] for t in range(len(expect))})
        training_op.run(feed_dict=feed_dict)
        
    print(loss.eval(feed_dict=feed_dict))

[ 55.52490997]
[ 45.62505341]
[ 50.34844208]
[ 49.02774429]
[ 45.09157562]
[ 53.64335632]
[ 42.15190125]
[ 45.09565735]
[ 40.51711655]
[ 39.62434006]
[ 42.93576813]
[ 36.71058655]
[ 30.19380569]
[ 30.20914459]
[ 30.45394897]
[ 28.07727051]
[ 26.55237579]
[ 24.03799057]
[ 23.85711479]
[ 26.63663101]
